In [27]:
import warnings
warnings.filterwarnings('ignore')
import datetime
import os
import gc
from collections import namedtuple
from pprint import pprint
from tqdm import tqdm

import numpy as np
import pandas as pd

import tensorboard
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
from torch.nn import CrossEntropyLoss
from torch.optim import AdamW
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau, CosineAnnealingLR, CyclicLR
import torchmetrics

from datasets import load_dataset, load_from_disk, concatenate_datasets, DatasetDict, Dataset
from transformers import get_linear_schedule_with_warmup
from transformers import DataCollatorForSeq2Seq
from transformers import BartConfig, T5Config
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
import evaluate
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge import Rouge

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

from utils import generate_seq, generate_input_target, generate_from_data, eval

In [2]:
tokenized_dataset_path = './data/gogamza-kobart-base-v2_tokenized_dataset'
baseline_checkpoint_path = '/home/thesol1/projects/KoGrammar/models/230510-16_32'

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
if os.path.exists(tokenized_dataset_path):
    tokenized_dataset = load_from_disk(tokenized_dataset_path)
    print(tokenized_dataset)
else:
    print("Tokenized dataset not found")

DatasetDict({
    train: Dataset({
        features: ['__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1016426
    })
    train_baseline: Dataset({
        features: ['__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 508213
    })
    train_distil: Dataset({
        features: ['__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 508212
    })
    valid: Dataset({
        features: ['__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 56468
    })
    test: Dataset({
        features: ['__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 56469
    })
})


## Baseline Test

In [5]:
tokenizer = AutoTokenizer.from_pretrained(baseline_checkpoint_path)
model = AutoModelForSeq2SeqLM.from_pretrained(baseline_checkpoint_path).to(device)
model.eval()

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(30000, 768, padding_idx=3)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(30000, 768, padding_idx=3)
      (embed_positions): BartLearnedPositionalEmbedding(1028, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=

In [6]:
validset = tokenized_dataset['valid'].shuffle()
test_sample = validset.shuffle().select(range(1))
test_sample

Dataset({
    features: ['__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 1
})

In [7]:
validset = tokenized_dataset['valid'].with_format("torch", device=device)
test_sample = validset.shuffle().select(range(1))
print(test_sample['input_ids'].shape)
output = generate_from_data(model, tokenizer, test_sample)

print(output['input_text'], output['generated_text'], output['target_text'], sep='\n\n')

torch.Size([1, 512])
누나랑 연락하는거 조아여!@@

누나랑 연락하는 거 좋아! @@

누나랑 연락하는 거 좋아요! @@


In [8]:
type(validset)

datasets.arrow_dataset.Dataset

In [30]:
accuracy_lt = []
bleu_score_lt = []
rouge_score_lt = []
accuracy = evaluate.load('accuracy')
bleu = evaluate.load('google_bleu')
#rouge = evaluate.load('rouge')
rouge = Rouge()

for example in tqdm(validset.shuffle().select(range(2000))):
    data = dict()
    for key in example:
        data[key] = example[key].unsqueeze(0)
    output = generate_from_data(model, tokenizer, data)
    generated_text = output['generated_text']
    target_text = output['target_text']

    try:
        #accuracy_score = accuracy.compute(predictions=generated_text, references=target_text, tokenizer=tokenizer)
        #bleu_score = bleu.compute(predictions=generated_text, references=target_text, tokenizer=tokenizer)
        bleu_score = sentence_bleu([target_text], generated_text, smoothing_function=SmoothingFunction().method1)
        rouge_score = rouge.get_scores(generated_text, target_text)[0]['rouge-2']['f']
        #rouge_score = rouge.compute(predictions=generated_text, references=target_text)
    except ValueError:
        continue

    #accuracy_lt.append(accuracy_score)
    bleu_score_lt.append(bleu_score)
    rouge_score_lt.append(rouge_score)

bleu_score_series = pd.Series(bleu_score_lt)
rouge_score_series = pd.Series(rouge_score_lt)
print(bleu_score_series.mean(), rouge_score_series.mean(), sep='\t')
result_df = pd.concat([bleu_score_series, rouge_score_series], axis=1)

100%|██████████| 1000/1000 [02:48<00:00,  5.95it/s]

0.7985125476607406
0.5424880093483264


TypeError: cannot concatenate object of type '<class 'numpy.float64'>'; only Series and DataFrame objs are valid

In [ ]:
result_df.mean()